In [1]:
import tensorflow as tf
import collections
import zipfile, random
import numpy as np

In [8]:
filename = 'text8.zip'

def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


- Total 17,005,207 words for training consisted of array of string.
- We need to convert this string array to digit array.

In [9]:
vocabulary_size = 20

def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count)

Most common words (+UNK) [['UNK', 11668632], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430), ('two', 192644), ('is', 183153), ('as', 131815), ('eight', 125285), ('for', 118445), ('s', 116710), ('five', 115789), ('three', 114775), ('was', 112807), ('by', 111831)]


- We choose the most frequent `vocabulary_size` words. 
- `UNK` means the rest of them except the most frequenty words.
- In this example, 
    1. UNK
    1. the
    1. of
    1. and
    1. one
    1. in
    1. a
    1. to
    1. zero
    1. nine
    1. two
    1. is
    1. as
    1. eight
    1. for
    1. s
    1. five
    1. three
    1. was
    1. by

In [10]:
print('Before data', words[:10])
print('After data', data[:10])
del words  # Hint to reduce memory.

Before data ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
After data [0, 0, 12, 6, 0, 2, 0, 0, 0, 0]


- The above result shows converted result from array of string to array of digits.
- Mapping between a word and a digit is the `dictionary`, that will appear below block.

In [12]:
print(dictionary)
print(reverse_dictionary)

{'and': 3, 'five': 16, 'the': 1, 'zero': 8, 'nine': 9, 'by': 19, 'a': 6, 'to': 7, 'two': 10, 'three': 17, 'one': 4, 'UNK': 0, 'as': 12, 'of': 2, 'in': 5, 'was': 18, 'for': 14, 's': 15, 'eight': 13, 'is': 11}
{0: 'UNK', 1: 'the', 2: 'of', 3: 'and', 4: 'one', 5: 'in', 6: 'a', 7: 'to', 8: 'zero', 9: 'nine', 10: 'two', 11: 'is', 12: 'as', 13: 'eight', 14: 'for', 15: 's', 16: 'five', 17: 'three', 18: 'was', 19: 'by'}


Function to generate a training batch for the skip-gram model.

This is used when we apply stochastic gradient method to skip gram model.

In [13]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['UNK', 'UNK', 'as', 'a', 'UNK', 'of', 'UNK', 'UNK']

with num_skips = 2 and skip_window = 1:
    batch: ['UNK', 'UNK', 'as', 'as', 'a', 'a', 'UNK', 'UNK']
    labels: ['UNK', 'as', 'a', 'UNK', 'UNK', 'as', 'a', 'of']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['UNK', 'a', 'UNK', 'UNK', 'as', 'UNK', 'UNK', 'of']
